Created a notebook so it can be organized. Started Aug 12, 2022

# Set up

In [6]:
#@title ## Base imports
import os
import sys
import numpy as np
import scipy
import sklearn
import sklearn.linear_model
import pandas as pd
idx = pd.IndexSlice
import IPython
import plotly
import plotly.express as px
import plotly.graph_objects as go
import textwrap
import collections

import IPython.display

import warnings
import requests
import urllib.request
import json
import copy   # to perform dict deep copy


<IPython.core.display.Javascript object>

In [2]:
#@title ## Option 1) Mount google drive and import my code

mountpoint_folder_name = "gdrive"  # can be anything, doesn't have to be "drive"
project_path_within_drive = "PythonProjects/GeospatialAnalysis" #@param {type:"string"}
project_path_full = os.path.join("/content/",mountpoint_folder_name,
                        "MyDrive",project_path_within_drive)
try:
    import google.colab.drive
    import os, sys
    # Need to move out of google drive directory if going to remount
    %cd
    # drive.mount documentation can be accessed via: drive.mount?
    #Signature: drive.mount(mountpoint, force_remount=False, timeout_ms=120000, use_metadata_server=False)
    google.colab.drive.mount(os.path.join("/content/",mountpoint_folder_name), force_remount=True)  # mounts to a folder called mountpoint_folder_name

    if project_path_full not in sys.path:
        pass
        #sys.path.insert(0,project_path_full)
    %cd {project_path_full}
    
except ModuleNotFoundError:  # in case not run in Google colab
    import traceback
    traceback.print_exc()

/root
Mounted at /content/gdrive
/content/gdrive/MyDrive/Computer Backups/Rahul Yerrabelli drive/PythonProjects/GeospatialAnalysis


In [ ]:
#@title ## Option 2) Clone project files from GitHub

!git clone https://github.com/ryerrabelli/GeospatialAnalysis.git

project_path_full = os.path.join("/content/","GeospatialAnalysis/")
sys.path.insert(1,project_path_full)
%cd GeospatialAnalysis
print(sys.path)

Cloning into 'GeospatialAnalysis'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 198 (delta 0), reused 3 (delta 0), pack-reused 192
Receiving objects: 100% (198/198), 31.41 MiB | 8.15 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/GeospatialAnalysis
['', '/content/GeospatialAnalysis', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


# Helper functions

In [3]:
colab_ip = %system hostname -I   # uses colab magic to get list from bash
colab_ip = colab_ip[0].strip()   # returns "172.28.0.12"
# Get most precent port name with !sudo lsof -i -P -n | grep LISTEN
colab_port = 9000                # could use 6000, 8080, or 9000

notebook_filename = filename = requests.get(f"http://{colab_ip}:{colab_port}/api/sessions").json()[0]["name"]

# Avoids scroll-in-the-scroll in the entire Notebook
def resize_colab_cell():
  display(IPython.display.Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 10000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)


#@markdown # get_path_to_save()
def get_path_to_save(file_prefix="", save_filename:str=None, save_in_subfolder:str=None, extension="png", create_folder_if_necessary=True):
    save_path = ["outputs",
                f"{notebook_filename.split('.',1)[0]}",  # use split to remove file extension
                ]
    if save_in_subfolder is not None:
        if isinstance(save_in_subfolder, (list, tuple, set, np.ndarray) ):
            save_path.append(**save_in_subfolder)
        else:  # should be a string then
            save_path.append(save_in_subfolder)
    save_path = os.path.join(*save_path)
    if not os.path.exists(save_path) and create_folder_if_necessary:
        os.makedirs(save_path)
    return os.path.join(save_path, file_prefix+save_filename+"."+extension)

#@markdown # save_df()
def save_df(df, file_name:str, ):
    df.to_pickle( get_path_to_save(save_filename=file_name, extension="pkl") )
    df.to_excel( get_path_to_save(save_filename=file_name, extension="xlsx") )
    df.to_csv( get_path_to_save(save_filename=file_name, extension="csv") )

#@markdown # save_figure()
def save_figure(fig, file_name:str, animated=False):
    """
    fig is of type plotly.graph_objs._figure.Figure,
    Requires kaleido to be installed
    """
    fig.write_html( get_path_to_save(save_filename=file_name, extension="html") )
    if not animated:
        fig.write_image( get_path_to_save(save_filename=file_name, extension="svg"))
        fig.write_image( get_path_to_save(save_filename=file_name, extension="png") )
        fig.write_image( get_path_to_save(save_filename=file_name, extension="jpg") )


years = np.arange(2015,2019+1)

#@markdown # calc_regression()
def calc_regression(y, x):
    import collections
    regress = scipy.stats.linregress(x, y=y)
    # R = Pearson coefficient
    # p indicates p-value
    # Use ordered dict to retain order
    return collections.OrderedDict({
        "Mean": np.mean(y, axis=0),
        "Sum": np.sum(y, axis=0),
        "Slope": regress.slope, 
        "SlopeSE": regress.stderr,
        "Intercept": regress.intercept, 
        "InterceptSE": regress.intercept_stderr,
        "R": regress.rvalue, 
        "p": regress.pvalue, 
         
        })


# CMS API access - currently not used

In [ ]:

# For the "Medicare Physician & Other Practitioners - by Provider and Service" database
# https://data.cms.gov/provider-summary-by-type-of-service/medicare-physician-other-practitioners/medicare-physician-other-practitioners-by-provider-and-service

CMS_dataset_uuids = {
    2019:	"5fccd951-9538-48a7-9075-6f02b9867868",
    2018:	"02c0692d-e2d9-4714-80c7-a1d16d72ec66",
    2017:	"7ebc578d-c2c7-46fd-8cc8-1b035eba7218",
    2016:	"5055d307-4fb3-4474-adbb-a11f4182ee35",
    2015:	"0ccba18d-b821-47c6-bb55-269b78921637",
    }

# Column categories
#https://data.cms.gov/resources/medicare-physician-other-practitioners-by-provider-and-service-data-dictionary
"""Rndrng_NPI	TEXT
Rndrng_Prvdr_Last_Org_Name	TEXT
Rndrng_Prvdr_First_Name	TEXT
Rndrng_Prvdr_MI	TEXT
Rndrng_Prvdr_Crdntls	TEXT
Rndrng_Prvdr_Gndr	TEXT
Rndrng_Prvdr_Ent_Cd	TEXT
Rndrng_Prvdr_St1	TEXT
Rndrng_Prvdr_St2	TEXT
Rndrng_Prvdr_City	TEXT
Rndrng_Prvdr_State_Abrvtn	TEXT
Rndrng_Prvdr_State_FIPS	TEXT
Rndrng_Prvdr_Zip5	TEXT
Rndrng_Prvdr_RUCA	TEXT
Rndrng_Prvdr_RUCA_Desc	TEXT
Rndrng_Prvdr_Cntry	TEXT
Rndrng_Prvdr_Type	TEXT
Rndrng_Prvdr_Mdcr_Prtcptg_Ind	TEXT
HCPCS_Cd	TEXT
HCPCS_Desc	TEXT
HCPCS_Drug_Ind	TEXT
Place_Of_Srvc	TEXT
Tot_Benes	NUMERIC
Tot_Srvcs	NUMERIC
Tot_Bene_Day_Srvcs	NUMERIC
Avg_Sbmtd_Chrg	NUMERIC
Avg_Mdcr_Alowd_Amt	NUMERIC
Avg_Mdcr_Pymt_Amt	NUMERIC
Avg_Mdcr_Stdzd_Amt	NUMERIC"""   


'Rndrng_NPI\tTEXT\nRndrng_Prvdr_Last_Org_Name\tTEXT\nRndrng_Prvdr_First_Name\tTEXT\nRndrng_Prvdr_MI\tTEXT\nRndrng_Prvdr_Crdntls\tTEXT\nRndrng_Prvdr_Gndr\tTEXT\nRndrng_Prvdr_Ent_Cd\tTEXT\nRndrng_Prvdr_St1\tTEXT\nRndrng_Prvdr_St2\tTEXT\nRndrng_Prvdr_City\tTEXT\nRndrng_Prvdr_State_Abrvtn\tTEXT\nRndrng_Prvdr_State_FIPS\tTEXT\nRndrng_Prvdr_Zip5\tTEXT\nRndrng_Prvdr_RUCA\tTEXT\nRndrng_Prvdr_RUCA_Desc\tTEXT\nRndrng_Prvdr_Cntry\tTEXT\nRndrng_Prvdr_Type\tTEXT\nRndrng_Prvdr_Mdcr_Prtcptg_Ind\tTEXT\nHCPCS_Cd\tTEXT\nHCPCS_Desc\tTEXT\nHCPCS_Drug_Ind\tTEXT\nPlace_Of_Srvc\tTEXT\nTot_Benes\tNUMERIC\nTot_Srvcs\tNUMERIC\nTot_Bene_Day_Srvcs\tNUMERIC\nAvg_Sbmtd_Chrg\tNUMERIC\nAvg_Mdcr_Alowd_Amt\tNUMERIC\nAvg_Mdcr_Pymt_Amt\tNUMERIC\nAvg_Mdcr_Stdzd_Amt\tNUMERIC'

In [ ]:

def access_CMS_data(years, query, print_checkpoints=True, max_length=100000, return_pandas=True):
    records = {}
    if years is None:
        years = CMS_dataset_uuids.keys()
    for year in years:
        uuid = CMS_dataset_uuids[year]
        url_data = f"https://data.cms.gov/data-api/v1/dataset/{uuid}/data"
        url_stats = f"{url_data}/stats"
        query_stats = copy.deepcopy(query)
        stats_response = requests.get(url_stats, params=query_stats)
        if stats_response.status_code == 200:
            stats_response_json = stats_response.json()
            found_rows_ct = stats_response_json["found_rows"] 
            total_rows_ct = stats_response_json["total_rows"]
            if print_checkpoints: print(stats_response_json)

            query_offset = 0
            query_size = query["size"]
            records[year] = []
            while query_offset < found_rows_ct:
                query_data = copy.deepcopy(query)
                data_response = requests.get(url_data, params=query_data)
                if data_response.status_code == 200:
                    data_response_json = data_response.json()
                    # append lists
                    if print_checkpoints and query_offset>0: print("query_offset", query_offset)
                    records[year] = records[year] + data_response_json
                query_offset += query_size
    
    if return_pandas:
        import pandas as pd
        return pd.concat([pd.DataFrame.from_dict(year_of_records) for year_of_records in records.values()], keys=records.keys())
    else:
        # return as dict of list
        return records



In [ ]:
uuid = CMS_dataset_uuids[2019]
query = {
    "column":"HCPCS_Cd,HCPCS_Desc,Tot_Benes", 
    #"group_by":"HCPCS_Cd",
    "offset":0, "size":10, "keyword":"30140"
    }
#url = f"https://data.cms.gov/provider-data/api/1/metastore/schemas/dataset/{uuid}/data?column=Rndrng_Prvdr_State_FIPS&offset=0&size=10"
url = f"https://data.cms.gov/data-api/v1/dataset/{uuid}/data"
#url = f"https://data.cms.gov/data-api/v1/dataset/{uuid}/data?column=Rndrng_Prvdr_State_FIPS&offset=0&size=10"
response = requests.get(url, params=query)

if response.status_code == 200:
    print(response.json())
    display(pd.DataFrame.from_dict(response.json()))

[{'HCPCS_Cd': '30140', 'HCPCS_Desc': 'Removal of nasal air passage', 'Tot_Benes': '13'}, {'HCPCS_Cd': '30140', 'HCPCS_Desc': 'Removal of nasal air passage', 'Tot_Benes': '16'}, {'HCPCS_Cd': '30140', 'HCPCS_Desc': 'Removal of nasal air passage', 'Tot_Benes': '21'}, {'HCPCS_Cd': '30140', 'HCPCS_Desc': 'Removal of nasal air passage', 'Tot_Benes': '16'}, {'HCPCS_Cd': '30140', 'HCPCS_Desc': 'Removal of nasal air passage', 'Tot_Benes': '16'}, {'HCPCS_Cd': '30140', 'HCPCS_Desc': 'Removal of nasal air passage', 'Tot_Benes': '12'}, {'HCPCS_Cd': '30140', 'HCPCS_Desc': 'Removal of nasal air passage', 'Tot_Benes': '13'}, {'HCPCS_Cd': '30140', 'HCPCS_Desc': 'Removal of nasal air passage', 'Tot_Benes': '15'}, {'HCPCS_Cd': '30140', 'HCPCS_Desc': 'Removal of nasal air passage', 'Tot_Benes': '60'}, {'HCPCS_Cd': '30140', 'HCPCS_Desc': 'Removal of nasal air passage', 'Tot_Benes': '13'}]


,HCPCS_Cd,HCPCS_Desc,Tot_Benes
0,30140,Removal of nasal air passage,13
1,30140,Removal of nasal air passage,16
2,30140,Removal of nasal air passage,21
3,30140,Removal of nasal air passage,16
4,30140,Removal of nasal air passage,16
5,30140,Removal of nasal air passage,12
6,30140,Removal of nasal air passage,13
7,30140,Removal of nasal air passage,15
8,30140,Removal of nasal air passage,60
9,30140,Removal of nasal air passage,13


In [ ]:
query = {
    #     "column":"HCPCS_Cd,HCPCS_Desc,Tot_Benes", 
    # 
    "column":"HCPCS_Cd,HCPCS_Desc,HCPCS_Drug_Ind,Place_Of_Srvc,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Alowd_Amt,Avg_Mdcr_Pymt_Amt,Avg_Mdcr_Stdzd_Am", 
    #"group_by":"HCPCS_Cd",
    "offset":0, "size":200, "keyword":"60240"
    }
records = access_CMS_data(None, query)

{'found_rows': 89, 'total_rows': 10140228}
query_offset 0
{'found_rows': 108, 'total_rows': 9961865}
query_offset 0
{'found_rows': 108, 'total_rows': 9847443}
query_offset 0
{'found_rows': 167, 'total_rows': 9714896}
query_offset 0
{'found_rows': 168, 'total_rows': 9496848}
query_offset 0


In [ ]:
display(records)

HCPCS_Cd          HCPCS_Desc HCPCS_Drug_Ind Place_Of_Srvc Tot_Benes  \
2019 0      60240  Removal of thyroid              N             F        18   
     1      60240  Removal of thyroid              N             F        13   
     2      60240  Removal of thyroid              N             F        16   
     3      60240  Removal of thyroid              N             F        13   
     4      60240  Removal of thyroid              N             F        21   
...           ...                 ...            ...           ...       ...   
2015 163    60240  Removal of thyroid              N             F        15   
     164    60240  Removal of thyroid              N             F        13   
     165    60240  Removal of thyroid              N             F        12   
     166    60240  Removal of thyroid              N             F        11   
     167    60240  Removal of thyroid              N             F        12   

         Tot_Srvcs Tot_Bene_Day_Srvcs Avg_Sbmtd_Chrg Avg_Mdcr_Alowd_Amt  \
2019 0          18                 18           4096                126   
     1          13                 13   5807.6923077       906.30846154   
     2          16                 16           1935         936.269375   
     3          13                 13   2375.8461538       826.54538462   
     4          21                 21   2689.7333333       819.35142857   
...            ...                ...            ...                ...   
2015 163        15                 15           2856       838.11933333   
     164        13                 13           4974       1021.3253846   
     165        12                 12           2500       961.99416667   
     166        11                 11   9500.1227273            2213.83   
     167        12                 12         2890.5             783.83   

         Avg_Mdcr_Pymt_Amt  
2019 0              100.39  
     1        724.50692308  
     2          739.449375  
     3        651.52153846  
     4        650.34190476  
...                    ...  
2015 163            659.96  
     164      813.74076923  
     165      766.34166667  
     166           1763.86  
     167      624.51666667  

[640 rows x 10 columns]

In [ ]:

save_df(records, "records_for_HCPCS=60240")

# Procedures analysis

## Load ENT procedures df from csv file
This is specifically a wide type df so it is one row per procedure with years as different columns.To understand what is meant by long type and wide type dataframes, see https://towardsdatascience.com/visualization-with-plotly-express-comprehensive-guide-eb5ee4b50b57

The slope given in the csv file is actually the inverse slope. We need to either recalculate it or invert it. I will just recalculate all the regression values.

In [16]:
df_procedures_orig = pd.read_csv("data/1_renamed/procedure_specific_data.csv",
                                 keep_default_na=False, # makes empty string cells still be interpreted as str 
                                 na_values=["nan", "NaN"],  # need to specify "NaN", not included by default
                                dtype={
                                    "Specialty": str,
                                    "Group": str,
                                    "HCPCS Code": str,
                                    "Total Number of Services": np.int64,
                                    **{f"Total Number of Services: {year}": np.int64 for year in range(2015,2019+1)},
                                    "% ASC Procedures": np.float64,
                                    "% ASC Billing": np.float64,
                                    })  # gets per healthcare code info
print(f"df_procedures_orig.shape = {df_procedures_orig.shape}")
df_procedures_orig.head(1)

<IPython.core.display.Javascript object>

df_procedures_orig.shape = (53, 22)


,Specialty,Group,HCPCS Code,HCPCS Description,Total Number of Services,Total Medicare Payment Amount,Total Number of Services: 2019,Total Medicare Payment Amount: 2019,Total Number of Services: 2018,Total Medicare Payment Amount: 2018,...,Total Number of Services: 2016,Total Medicare Payment Amount: 2016,Total Number of Services: 2015,Total Medicare Payment Amount: 2015,Total Medicare Payment Amount: Slope,Total Medicare Payment Amount: Pearson Coef,Total Number of Services: Slope,Total Number of Services: Pearson Coef,% ASC Procedures,% ASC Billing
0,Facial plastics,,11042,Removal of skin and tissue first 20 sq cm or less,46073,4237372.2,6537,644733.1,11483,1317431.9,...,8266,600263.11,5179,509299.24,0.000002,0.425707,0.000101,0.245111,0.7767,0.893


## Clean df and recalculate regression

In [26]:
df_procedures_clean = df_procedures_orig.set_index(["Specialty","Group","HCPCS Code", "HCPCS Description"])

# Remove the "amount" word 
df_procedures_clean.columns = [col.replace("Total Medicare Payment Amount","Total Medicare Payment") for col in df_procedures_clean.columns]
# Make % column names match the rest of the df column names
df_procedures_clean.columns = [col.replace("ASC Billing","ASC Payment") for col in df_procedures_clean.columns]
df_procedures_clean.columns = [col.replace("ASC Procedures","ASC Services") for col in df_procedures_clean.columns]

# Drop columns besides the individual year ones. Will recalculate the other ones as a quality assurance check.
df_procedures_clean = df_procedures_clean.drop(columns=[
    col for col in df_procedures_clean.columns 
    if (("slope" in col.lower() or "pearson" in col.lower() or ":" not in col) and "%" not in col)
    ] )

# Rename the columns so they can be split  easier. The 20 is the first two digits of the year columns (e.g. "2019") 
df_procedures_clean.columns = [col.replace(": ",": : ").replace(": 20","Annual: 20") for col in df_procedures_clean.columns]
df_procedures_clean.columns = [ (col.replace("% ASC ","ASC: ") + ": %" if "% ASC" in col else col) for col in df_procedures_clean.columns]

# Make Multiindex
df_procedures_clean.columns = pd.MultiIndex.from_tuples([tuple(col.split(": ")) if ":" in col else (col,"","") for col in df_procedures_clean.columns], names=["Category","Stat","Year"])
df_procedures_clean = df_procedures_clean[sorted(df_procedures_clean)]  # rearrange cols alphabetically

col_categories = df_procedures_clean.columns.levels[0]  #["ASC", "Total Number of Services", "Total Medicare Payment"]

# Make aggregates across the specialties and the groups
"""specialties = df_procedures_clean.index.unique(level="Specialty")
all_groups = df_procedures_clean.index.unique(level="Group")
df_procedures_clean.loc[("Total",None,"Total","Total")] = df_procedures_clean.sum()
for specialty in specialties:
    df_procedures_clean.loc[(specialty,None,"Total","Total")]=df_procedures_clean.loc[specialty].sum()
    groups = df_procedures_clean.loc[specialty].index.unique(level="Group")
    for group in groups:
        if df_procedures_clean.loc[(specialty,group)].shape[0] > 1:
            df_procedures_clean.loc[(specialty,group,"Total","Total")]=df_procedures_clean.loc[(specialty,group)].sum()"""

#df_procedures_clean = df_procedures_clean.sort_index()

<IPython.core.display.Javascript object>

'specialties = df_procedures_clean.index.unique(level="Specialty")\nall_groups = df_procedures_clean.index.unique(level="Group")\ndf_procedures_clean.loc[("Total",None,"Total","Total")] = df_procedures_clean.sum()\nfor specialty in specialties:\n    df_procedures_clean.loc[(specialty,None,"Total","Total")]=df_procedures_clean.loc[specialty].sum()\n    groups = df_procedures_clean.loc[specialty].index.unique(level="Group")\n    for group in groups:\n        if df_procedures_clean.loc[(specialty,group)].shape[0] > 1:\n            df_procedures_clean.loc[(specialty,group,"Total","Total")]=df_procedures_clean.loc[(specialty,group)].sum()'

In [24]:
df_procedures_clean.head()

<IPython.core.display.Javascript object>

Category                                                                                ASC  \
Stat                                                                                Payment   
Year                                                                                      %   
Specialty       Group HCPCS Code HCPCS Description                                            
Facial plastics       11042      Removal of skin and tissue first 20 sq cm or less   0.8930   
Head & neck     G     11642      Removal of malignant growth (1.1 to 2.0 centime...  0.7798   
                      11643      Removal of malignant growth (2.1 to 3.0 centime...  0.8421   
Facial plastics A     14040      Tissue transfer repair of wound (10 sq centimet...  0.8737   
                      14041      Tissue transfer repair of wound (10.1 to 30.0 s...  0.8563   

Category                                                                                      \
Stat                                                                                Services   
Year                                                                                       %   
Specialty       Group HCPCS Code HCPCS Description                                             
Facial plastics       11042      Removal of skin and tissue first 20 sq cm or less    0.7767   
Head & neck     G     11642      Removal of malignant growth (1.1 to 2.0 centime...   0.7914   
                      11643      Removal of malignant growth (2.1 to 3.0 centime...   0.8869   
Facial plastics A     14040      Tissue transfer repair of wound (10 sq centimet...   0.8321   
                      14041      Tissue transfer repair of wound (10.1 to 30.0 s...   0.8406   

Category                                                                            Total Medicare Payment  \
Stat                                                                                                Annual   
Year                                                                                                  2015   
Specialty       Group HCPCS Code HCPCS Description                                                           
Facial plastics       11042      Removal of skin and tissue first 20 sq cm or less               509299.24   
Head & neck     G     11642      Removal of malignant growth (1.1 to 2.0 centime...              289427.10   
                      11643      Removal of malignant growth (2.1 to 3.0 centime...              139924.42   
Facial plastics A     14040      Tissue transfer repair of wound (10 sq centimet...             4175118.98   
                      14041      Tissue transfer repair of wound (10.1 to 30.0 s...             2481173.43   

Category                                                                                         \
Stat                                                                                              
Year                                                                                       2016   
Specialty       Group HCPCS Code HCPCS Description                                                
Facial plastics       11042      Removal of skin and tissue first 20 sq cm or less    600263.11   
Head & neck     G     11642      Removal of malignant growth (1.1 to 2.0 centime...   245483.24   
                      11643      Removal of malignant growth (2.1 to 3.0 centime...   140722.56   
Facial plastics A     14040      Tissue transfer repair of wound (10 sq centimet...  3907715.69   
                      14041      Tissue transfer repair of wound (10.1 to 30.0 s...  2174377.77   

Category                                                                                         \
Stat                                                                                              
Year                                                                                       2017   
Specialty       Group HCPCS Code HCPCS Description                                                


In [ ]:
df_procedures_clean2 = df_procedures_clean.sort_index(level=[0,1,2,3])
df_procedures_clean2.index.is_monotonic_increasing

#df_procedures_clean.loc[(specialty,slice(None),slice("0","9"),slice(None))]
#df_procedures_clean.loc[idx[specialty,:,"31571":"31622"]]
#df_procedures_clean.loc[(specialty,"B","a","a"):tuple()]

False

In [66]:
col_category.split(" ")[-1]

<IPython.core.display.Javascript object>

'Services'

In [84]:
# Calculate regression and sum and mean from individual year later
df_procedures_recalc = df_procedures_clean.copy()


# Convert columns with percentages (i.e. ASC %) into absolute numbers so can aggregate properly
for col_category in col_categories:
    if col_category in df_procedures_recalc and "Annual" in df_procedures_recalc[col_category].columns:
        mean_df = df_procedures_recalc[(col_category,"Annual")].mean(axis=1)
        percent_col_name = col_category.split(" ")[-1]
        mean_ASC_df = mean_df * df_procedures_recalc[("ASC",percent_col_name,"%")]
        mean_HOPD_df = mean_df * (1-df_procedures_recalc[("ASC",percent_col_name,"%")])
        df_procedures_recalc[(col_category,"ASC","Mean")]=mean_ASC_df
        df_procedures_recalc[(col_category,"HOPD","Mean")]=mean_HOPD_df
        #df_procedures_recalc[(col_category,"","Slope")]=df_procedures_recalc[(col_category,"Annual")].apply(calc_regression,axis=1)

# Drop columns with percentages for now since those can't be easily aggregated (i.e. you can't just average them directly, you have to weight them)
df_procedures_recalc = df_procedures_recalc.drop(columns=[
    col for col in df_procedures_clean.columns  if "%" in col
])


<IPython.core.display.Javascript object>

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:2854: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [94]:
# Make aggregates (totals) across the specialties and the groups
specialties = df_procedures_recalc.index.unique(level="Specialty")
all_groups = df_procedures_recalc.index.unique(level="Group")
df_procedures_recalc.loc[("Total",None,"Total","Total")] = df_procedures_recalc.sum()
for specialty in specialties:
    df_procedures_recalc.loc[(specialty,None,"Total","Total")]=df_procedures_recalc.loc[specialty].sum()
    groups = df_procedures_recalc.loc[specialty].index.unique(level="Group")
    for group in groups:
        if df_procedures_recalc.loc[(specialty,group)].shape[0] > 1:
            df_procedures_recalc.loc[(specialty,group,"Total","Total")]=df_procedures_recalc.loc[(specialty,group)].sum()

# Calculate overall statistics (mean, SE, p value, R, etc)
for col_category in col_categories:
    if col_category in df_procedures_recalc and "Annual" in df_procedures_recalc[col_category].columns:
        new_df = df_procedures_recalc[(col_category,"Annual")].apply(calc_regression,axis=1, result_type="expand", args=(years,) )
        df_procedures_recalc[[(col_category,"Overall",new_col) for new_col in new_df.columns ]]=new_df
        #df_procedures_recalc[(col_category,"","Slope")]=df_procedures_recalc[(col_category,"Annual")].apply(calc_regression,axis=1)

# Recalculate ASC %. This should match the % before analysis for the individual procedures, but now has the correct info for aggreageted procedures
for col_category in col_categories:
    if col_category in df_procedures_recalc and "Annual" in df_procedures_recalc[col_category].columns:
        df_procedures_recalc[(col_category,"ASC","%")] = \
            df_procedures_recalc[(col_category,"ASC","Mean")] / df_procedures_recalc[(col_category,"Overall","Mean")]
        df_procedures_recalc[(col_category,"HOPD","%")] = \
            df_procedures_recalc[(col_category,"HOPD","Mean")] / df_procedures_recalc[(col_category,"Overall","Mean")]


# Rearrange cols alphabetically, but only by the first two elements of the each column's name tuple
# This allows the order of the newly added columns to remain relative to themselves, but be rearranged relative to the other columns
#df_procedures_recalc = df_procedures_recalc[sorted(df_procedures_recalc.columns, key=(lambda x: x[0:2]))]  
# Alternatively, rearrange by first level alphabetically, then length of second level 
df_procedures_recalc = df_procedures_recalc[sorted(df_procedures_recalc.columns, key=(lambda x: (x[0],len(x[1])) ) )]  

#df_procedures = df_procedures.sort_values(by=("Total Number of Services","","Sum"), ascending=False)  # sort rows by volume 
df_procedures_recalc = df_procedures_recalc.sort_values(by=("Total Medicare Payment","Overall","Mean"), ascending=False)  # sort rows by volume 
df_procedures_recalc = df_procedures_recalc.sort_index()

<IPython.core.display.Javascript object>

<ipython-input-94-f5466e046238>:9: PerformanceWarning:

indexing past lexsort depth may impact performance.

<ipython-input-94-f5466e046238>:10: PerformanceWarning:

indexing past lexsort depth may impact performance.



## Save recalculated procedures

In [96]:
with pd.option_context("display.float_format", "{:,.2f}".format):
    display(df_procedures_recalc)

save_df(df_procedures_recalc, "df_procedures_recalc")

<IPython.core.display.Javascript object>

Category                                                                            Total Medicare Payment  \
Stat                                                                                                   ASC   
Year                                                                                                  Mean   
Specialty       Group HCPCS Code HCPCS Description                                                           
Facial plastics       11042      Removal of skin and tissue first 20 sq cm or less              756,794.67   
                      15004      Preparation of graft site of face, scalp, eyeli...             255,840.48   
                      15120      Skin graft of face, scalp, eyelids, mouth, neck...             267,342.90   
                      15260      Relocation of patient skin to nose, ears, eyeli...           3,427,951.78   
                      15730      Creation of flap graft to midface                              111,175.58   
...                                                                                                    ...   
Rhinology       J     31267      Removal of nasal sinus tissue using an endoscope             3,610,783.76   
                      31288      Removal of nasal sinus tissue using an endoscope               330,980.12   
                      Total      Total                                                       27,592,347.12   
                NaN   Total      Total                                                      298,758,984.65   
Total           NaN   Total      Total                                                    3,365,271,836.40   

Category                                                                                  \
Stat                                                                                       
Year                                                                                   %   
Specialty       Group HCPCS Code HCPCS Description                                         
Facial plastics       11042      Removal of skin and tissue first 20 sq cm or less  0.89   
                      15004      Preparation of graft site of face, scalp, eyeli... 0.44   
                      15120      Skin graft of face, scalp, eyelids, mouth, neck... 0.77   
                      15260      Relocation of patient skin to nose, ears, eyeli... 0.88   
                      15730      Creation of flap graft to midface                  0.72   
...                                                                                  ...   
Rhinology       J     31267      Removal of nasal sinus tissue using an endoscope   0.83   
                      31288      Removal of nasal sinus tissue using an endoscope   0.76   
                      Total      Total                                              0.82   
                NaN   Total      Total                                              0.60   
Total           NaN   Total      Total                                              0.63   

Category                                                                                              \
Stat                                                                                            HOPD   
Year                                                                                            Mean   
Specialty       Group HCPCS Code HCPCS Description                                                     
Facial plastics       11042      Removal of skin and tissue first 20 sq cm or less         90,679.77   
                      15004      Preparation of graft site of face, scalp, eyeli...       325,879.57   
                      15120      Skin graft of face, scalp, eyelids, mouth, neck...        78,150.64   
                      15260      Relocation of patient skin to nose, ears, eyeli...       470,549.05   
                      15730      Creation of flap graft to midface                         43,192.07   
...                             

## Some stats

## Plot procedures in response to Reviewer 1's response (in round #2)

In [338]:
def customize_bar_chart(fig: plotly.graph_objs.Figure):
    fig.update_layout(
        template="simple_white",
        font=dict(
                family="Arial",
                size=16,
                color="black",
            ),
    )
    # Below statements can be done in fig.update_layout(), but doing for_each allows it to work for each subplot when there are s
    fig.for_each_xaxis(lambda axis: axis.update(dict(
        zeroline=True,
        showgrid=True,
        mirror="ticks",
        gridcolor="#DDD",
        showspikes=True, spikemode="across", spikethickness=2, spikedash="solid"
    )))
    fig.for_each_yaxis(lambda axis: axis.update(dict(
        zeroline=True,
        showgrid=True,
        mirror="ticks",
        gridcolor="#DDD",
        showspikes=True, spikemode="across", spikethickness=2, spikedash="solid"
    )))

var_labels = {
    "Total Medicare Payment-Overall-Mean": "Total Medicare Payment - 5yr mean ($/yr)",
    **{f"Total Medicare Payment-Annual-{yr}": f"Total Medicare Payment - {yr} ($/yr)" for yr in range(2015,2019+1)},
    "Total Medicare Payment-ASC-Mean": "Medicare Payment - ASC", 
    "Total Medicare Payment-HOPD-Mean": "Medicare Payment - HOPD",
    "Otology, Total Medicare Payment-HOPD-Mean": "Medicare Payment - HOPD",
}
category_orders = {"Specialty": ["Facial plastics","Head & neck","Otology","Rhinology","Laryngology"]}
color_discrete_map = {
    "Total Medicare Payment-ASC-Mean": "black",
    "Total Medicare Payment-HOPD-Mean": "gray",
}
color_discrete_sequence = px.colors.qualitative.Safe

<IPython.core.display.Javascript object>

In [339]:
df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.sort_values(by="Total Medicare Payment-Overall-Mean").reset_index()
df["HCPCS"] = ("<b>#" + df["HCPCS Code"] + "</b>")
fig = px.bar(df, 
             x="HCPCS", 
             y=("Total Medicare Payment-Overall-Mean"),
             color="Specialty", text_auto=".2s",
             category_orders=category_orders, labels=var_labels,
             color_discrete_sequence=color_discrete_sequence,
             )
customize_bar_chart(fig)
fig.show()

<IPython.core.display.Javascript object>

In [340]:
df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.sort_values(by="Total Medicare Payment-Overall-Mean").reset_index()
df["HCPCS"] = ("<b>#" + df["HCPCS Code"] + "</b>")
fig = px.bar(df, 
             x="HCPCS", 
             y=("Total Medicare Payment-Overall-Mean"),
             color="Specialty",
             facet_col="Specialty",
             facet_col_wrap=2, 
             facet_row_spacing=0.1, # default is 0.07 when facet_col_wrap is used
             facet_col_spacing=0.05, # default is 0.03
             text_auto='.2s', 
             category_orders=category_orders, labels=var_labels,
             color_discrete_sequence=color_discrete_sequence,
             )
fig.update_xaxes(matches=None)
#fig.update_yaxes(matches=None)
customize_bar_chart(fig)
fig.for_each_xaxis(lambda axis: axis.update(dict(showticklabels=True, tickfont=dict(size=12))))
fig.for_each_yaxis(lambda axis: axis.update(dict(showticklabels=True, title="")))

#fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape=""))) if trace.name == "None" else (), )
fig.update_layout(
    width=1500, height=600,
    margin=dict(l=20, r=20, t=20, b=20),)
fig.for_each_annotation(lambda ann: ann.update(text=ann.text.split("=")[-1]))
fig.show()

<IPython.core.display.Javascript object>

In [341]:
df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.sort_values(by="Total Medicare Payment-Overall-Mean").reset_index()
df["HCPCS"] = ("<b>#" + df["HCPCS Code"] + "</b>")
fig = px.bar(df, 
             x="HCPCS", 
             y=[f"Total Medicare Payment-Annual-{yr}" for yr in range(2015,2019+1)],
             color="Specialty", text_auto='.2s',
             category_orders=category_orders, labels={**var_labels, "variable":"Year", "value":"Total Medicare Payment (USD)"},
             color_discrete_sequence=color_discrete_sequence,
             )
customize_bar_chart(fig)
fig.show()

<IPython.core.display.Javascript object>

In [343]:
df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.sort_values(by=["Specialty","Total Medicare Payment-Overall-Mean"])
df = df.reset_index()
df["HCPCS"] = ("<b>#" + df["HCPCS Code"] + "</b>")
fig = px.bar(df, 
             x="HCPCS", 
             y=["Total Medicare Payment-ASC-Mean", "Total Medicare Payment-HOPD-Mean"],
             color="Specialty",
             text_auto='.2s',
             pattern_shape_sequence=["","/"],
             color_discrete_sequence=color_discrete_sequence,
             pattern_shape="variable",  # variable and value are special words: https://plotly.com/python/wide-form/#assigning-inferred-columns-to-nondefault-arguments
             category_orders=category_orders, labels={**var_labels, "variable":"ASC/HOPD", "value":"Annual Medicare Payment (USD/yr)"},
             )
customize_bar_chart(fig)
fig.show()

<IPython.core.display.Javascript object>

In [254]:
a=df["Specialty"].unique()
a.sort()
a

<IPython.core.display.Javascript object>

array(['Facial plastics', 'Head & neck', 'Laryngology', 'Otology',
       'Rhinology'], dtype=object)

In [383]:
color_discrete_map = {
    "Total Medicare Payment-ASC-Mean": "#ccc",
    "Total Medicare Payment-HOPD-Mean": "#000",
}
pattern_shape_map = {
    "Total Medicare Payment-ASC-Mean": "/",
    "Total Medicare Payment-HOPD-Mean": "",
}

df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.sort_values(by=["Specialty","Total Medicare Payment-Overall-Mean"])
df = df.reset_index()
df["HCPCS"] = ("<b>#" + df["HCPCS Code"] + "</b>")
fig = px.bar(df, 
             x="HCPCS", 
             y=["Total Medicare Payment-HOPD-Mean","Total Medicare Payment-ASC-Mean"],
             color="variable", color_discrete_map=color_discrete_map,
             pattern_shape_map=pattern_shape_map,
             color_discrete_sequence=color_discrete_sequence,
             pattern_shape="variable",  # variable and value are special words: https://plotly.com/python/wide-form/#assigning-inferred-columns-to-nondefault-arguments
             category_orders=category_orders, labels={**var_labels, "variable":"ASC/HOPD", "value":"Annual Medicare Payment (USD/yr)"},
             )
customize_bar_chart(fig)

# https://stackoverflow.com/a/73313404/2879686
fig.for_each_trace(
    lambda trace: trace.update(
        text=(df["Total Medicare Payment-ASC-%"].apply("{:.1%}".format) if "ASC-" in trace.name else "" ), textfont=dict(color="#000"),
        textposition="auto",  marker_line_width=2, marker_line_color="#111"
    )
)
fig.update_layout(
    width=2000, height=400,
    margin=dict(l=20, r=20, t=20, b=20),)

specialty_freqs = df["Specialty"].value_counts().sort_index()  # sort index converts from being ordered by frequency to being ordered alphabetically
loc_x = 0
for ind, specialty in enumerate(specialty_freqs.index):
    fig.add_annotation(
        text=f"<b>{specialty}</b>", 
        width=15*specialty_freqs[specialty],
        x=loc_x-0.5+specialty_freqs[specialty]/2, 
        xanchor="center", axref="x", xref="x",
        bgcolor=color_discrete_sequence[ind], borderwidth=2, bordercolor="#000",
        font=dict(color= "#000" if ind<=2 else "#FFF"),
        #width=specialty_freqs[specialty],
        y=7e6, yanchor="bottom",
        showarrow=False,
                )

    fig.add_vline(x=loc_x-0.5, line_width=2, line_color="#000",opacity=1)
    fig.add_vrect(x0=loc_x-0.5, x1=loc_x + specialty_freqs[specialty]-0.5, opacity=0.1, fillcolor=color_discrete_sequence[ind])
    loc_x += specialty_freqs[specialty]


fig.show()

<IPython.core.display.Javascript object>

In [261]:
df["Specialty"].value_counts().sort_index().index

<IPython.core.display.Javascript object>

Index(['Facial plastics', 'Head & neck', 'Laryngology', 'Otology',
       'Rhinology'],
      dtype='object')

In [211]:
[trace.name for trace in fig.data]

<IPython.core.display.Javascript object>

['Total Medicare Payment-ASC-Mean', 'Total Medicare Payment-HOPD-Mean']

In [ ]:
df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.sort_values(by="Total Medicare Payment-Overall-Mean").reset_index()
df["HCPCS"] = ("<b>#" + df["HCPCS Code"] + "</b>")
fig = px.bar(df, 
             x="Specialty", 
             y="Total Medicare Payment-Overall-Mean",
             color="HCPCS Code",
             barmode="stack",
             category_orders=category_orders, labels=var_labels,
             text="HCPCS",
             )
fig.update_traces(textposition='inside')
customize_bar_chart(fig)
fig.show()

In [ ]:
split_text = "<br>".join(textwrap.wrap('This is a very long title and it would be great to have it on three lines', 
                            width=30))
func = lambda string: "<br>".join(textwrap.wrap(string, width=50))
split_text

'This is a very long title and<br>it would be great to have it<br>on three lines'

In [ ]:
df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.sort_values(by="Group")
df = df.reset_index()
stack = [f"X{ind}" for ind in range(100)]
stack.reverse()
df["Group"] = df["Group"].apply(lambda val: stack.pop() if val == "" else val)

df["HCPCS"] = ("<b>#" + df["HCPCS Code"] + "</b> " + df["HCPCS Description"]).apply(func)

fig = px.bar(df, 
             x="Specialty", 
             y="Total Medicare Payment-Overall-Mean",
             color="Group",
             barmode="stack",
             category_orders=category_orders, labels=var_labels,
             text="HCPCS",
             )
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.update_traces(textposition='inside')
customize_bar_chart(fig)
fig.show()

In [ ]:
df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.reset_index()
df["HCPCS"] = ("<b>#" + df["HCPCS Code"] + "</b>")


fig = px.pie(df, names='Specialty', values='Total Medicare Payment-Overall-Mean')
fig.show()

In [ ]:
df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.reset_index()
df["HCPCS"] = ("<b>#" + df["HCPCS Code"] + "</b>")


fig = px.pie(df, names='Specialty', 
             values='Total Medicare Payment-Overall-Mean'
)
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.show()


In [ ]:
df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.reset_index()
df = df.sort_values(by="Group")
df["HCPCS"] = ("#" + df["HCPCS Code"] )
stack = [f"X{ind}" for ind in range(100)]
stack.reverse()
#df["Group"] = df["Group"].apply(lambda val: stack.pop() if val == "" else val)
#df["Group"] = df["Group"].apply(lambda val: None if val == "" else val)
df["Group"] = df["Group"].apply(lambda val: "Other" if val == "" else val)
fig = px.sunburst(df, path=["Specialty","Group","HCPCS"], values="Total Medicare Payment-Overall-Mean",
                  #color="Group",#color_continuous_scale='RdBu',
                  labels={"A":""},
                  )
fig.update_traces(textinfo="label+percent entry", selector=dict(type='sunburst'))

fig.update_traces(insidetextorientation="radial")
fig.show()


ValueError: ignored

In [ ]:
df = df_procedures_recalc.loc[df_procedures_recalc.index.get_level_values(2) != "Total"]
df.columns = ["-".join(col) for col in df.columns]
df = df.reset_index()
df["HCPCS"] = ("<b>#" + df["HCPCS Code"] + "</b>")
fig = px.sunburst(df, path=["Specialty","Group","HCPCS Code"], values="Total Medicare Payment-Overall-Mean")
fig.show()

# County analysis

## Load data

In [ ]:
# @title Load spatial coordinates of counties
# Below is necessary for plotting chloropleths. 
with urllib.request.urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
# @title Load conversion df between FIPS code and county string
fips2county = pd.read_csv("data/fips2county.tsv", sep="\t", comment='#', dtype=str)

In [ ]:
# @title Load our ENT df of all counties, their info, and the Moran's analysis
# The ent CSV file only contains the counties which are analyzable
df_counties_wide_orig = pd.read_csv("data/1_renamed/county_specific_data.csv", dtype={"FIPS": str})

In [ ]:
df_counties_wide_orig.columns

Index(['FIPS', 'Total Number of Services', 'Total Medicare Payment Amount',
       'Total Number of Services: 2019', 'Total Medicare Payment Amount: 2019',
       'Total Number of Services: 2018', 'Total Medicare Payment Amount: 2018',
       'Total Number of Services: 2017', 'Total Medicare Payment Amount: 2017',
       'Total Number of Services: 2016', 'Total Medicare Payment Amount: 2016',
       'Total Number of Services: 2015', 'Total Medicare Payment Amount: 2015',
       'tot_ratio', '% ASC Procedures: 2019', '% ASC Billing: 2019',
       '% ASC Procedures: 2018', '% ASC Billing: 2018',
       '% ASC Procedures: 2017', '% ASC Billing: 2017',
       '% ASC Procedures: 2016', '% ASC Billing: 2016',
       '% ASC Procedures: 2015', '% ASC Billing: 2015', '% ASC Procedures',
       '% ASC Billing', 'Beneficiaries with Part A and Part B', 'Average Age',
       'Percent Male', 'Percent Non-Hispanic White',
       'Percent African American', 'Percent Hispanic',
       'Percent Eligible

In [ ]:
# @title Merge with the fips 2 county standard data set
df_counties_wide = pd.merge(left=df_counties_wide_orig, right=fips2county, how="left", left_on='FIPS', right_on='CountyFIPS')
# Insert a county "County, ST" col (i.e. "Monmouth, NJ" or "Champaign, IL") for ease
df_counties_wide.insert(1, "County_St", df_counties_wide["CountyName"].astype(str) + ", " + df_counties_wide["StateAbbr"].astype(str))

cols_renamed={
    "Average Age": "Average Age (years)",
    'Percent Male': "% Male",
    'Percent Non-Hispanic White': "% Non-Hispanic White",
    'Percent African American': "% African American",
    'Percent Hispanic': "% Hispanic",
    'Percent Eligible for Medicaid': "% Eligible for Medicaid",
    'pct_poverty': "% Poverty",
    'median_house_income': "Median Household Income",
    "Pct_wthout_high_diploma": "% without High School Graduation",
    'unemployment': "Unemployment Rate",
    'pct_uninsured': "% Uninsured",
    'tabacco': "% Tobacco Use",
    'obesity': "% Obesity",
    #"Asthma": "% with Asthma",
    '2013_Rural_urban_cont_code': "RUCA",
    'pop': "Overall Population",
    'Beneficiaries with Part A and Part B': "Medicare Beneficiaries Population",
    'Population Density': "Overall Population Density",
    'Medicare Population Density': "Medicare Population Density",
    "Moran I score for ACS billing fraction":  "Moran I for ASC billing fraction",  # It is "ASC" not "ACS"
}
df_counties_wide = df_counties_wide.rename(columns=cols_renamed)


In [ ]:
info_simple = ["FIPS", "CountyName","StateAbbr", "% ASC Billing"]
info_main = ["FIPS", "County",	"StateFIPS", "Total Medicare Payment Amount", "% ASC Procedures", "% ASC Billing",	"CountyFIPS_3",	"CountyName",	"StateName",	"CountyFIPS",	"StateAbbr",	"STATE_COUNTY", "Moran I for ASC billing fraction"]

df_counties_wide_simple=df_counties_wide[info_simple]
df_counties_wide_main=df_counties_wide[info_main]

# Display with all the columns
with pd.option_context('display.max_rows', 3, 'display.max_columns', None): 
    display(df_counties_wide_simple)


,FIPS,CountyName,StateAbbr,% ASC Billing
0,01017,Chambers,AL,0.000000
...,...,...,...,...
940,51041,Chesterfield,VA,47.027037


## Create analysis of which states were included

In [ ]:
counties_with_datas_counts = df_counties_wide.groupby(["StateAbbr"])["StateAbbr"].count()
counties_with_datas_pop = df_counties_wide.groupby(["StateAbbr"])["Overall Population"].sum()
all_counties_counts = fips2county.groupby(["StateAbbr"])["StateAbbr"].count()

In [ ]:
df_counties_by_state_and_Moran = df_counties_wide.groupby(["StateAbbr","Moran I for ASC billing fraction"])["StateAbbr"].count()
df_counties_by_state_and_Moran = pd.DataFrame(df_counties_by_state_and_Moran).rename(columns={"StateAbbr":"#"}).reset_index().pivot(index="StateAbbr",columns="Moran I for ASC billing fraction",values="#")
save_df(df_counties_by_state_and_Moran, "df_counties_by_state_and_Moran")

In [ ]:
df_counties_by_state = pd.DataFrame({"Included":counties_with_datas_counts, "All Counties":all_counties_counts}, dtype="Int64")
df_counties_by_state["Ratio"] = df_counties_by_state["Included"]/df_counties_by_state["All Counties"]
df_counties_by_state["Included total population"] = counties_with_datas_pop
df_counties_by_state = df_counties_by_state.sort_values(by="Ratio",ascending=False)
display(df_counties_by_state)
save_df(df_counties_by_state, "df_counties_by_state")

,Included,All Counties,Ratio,Included total population
StateAbbr,,,,
DE,3,3,1.0,957247.8
NJ,20,21,0.952381,8815513.4
RI,4,5,0.8,1008467.0
CT,6,8,0.75,3260957.6
MA,10,14,0.714286,6590463.4
PA,47,67,0.701493,12050026.8
NH,7,10,0.7,1225141.0
NY,40,62,0.645161,18243240.6
CA,37,58,0.637931,38322360.6


In [ ]:
df_counties_wide_main[["County","StateAbbr","Moran I for ASC billing fraction"]]

,County,StateAbbr,Moran I for ASC billing fraction
0,Chambers,AL,Non Significant
1,Colbert,AL,Non Significant
2,Dale,AL,Non Significant
3,Limestone,AL,Non Significant
4,White,AR,Non Significant
...,...,...,...
936,Franklin,KY,Non Significant
937,Laramie,WY,Non Significant
938,Lewis,WV,Low-Low
939,Windsor,VT,Low-Low


## Create long df from wide df- i.e. separate out the year columns into different rows

In [ ]:
col_categories = ["Total Number of Services:", "Total Medicare Payment Amount:", "% ASC Procedures:", "% ASC Billing:"]
cols_to_keep = ["FIPS","County_St"]  # columns to keep in every subgroup so you can line up extra info later

# Create list of df's to combine later, each df is from melting of one col_category of columns
df_counties_longs = []

# Convert each type of col_category to long format in separate dataframes
for col_category in col_categories:
        df_counties_long = df_counties_wide.melt(id_vars=cols_to_keep, 
                               var_name="Year", 
                               value_vars=[f"{col_category} {year}" for year in range(2015, 2019 +1)], 
                               value_name=f"{col_category} in Year",
                               )
        df_counties_long["Year"] = df_counties_long["Year"].replace({ f"{col_category} {year}":f"{year}" for year in range(2015, 2019 +1)})
        df_counties_longs.append(df_counties_long)

# Merge the separate col_category dataframes
df_counties_long = df_counties_longs[0]
for ind in range(1,len(df_counties_longs)):
    df_counties_long = pd.merge(left=df_counties_long, right=df_counties_longs[ind], how="outer", on=(cols_to_keep+["Year"]) )

# Merge with the overall wide dataframe to keep those other values
df_counties_long = pd.merge(left=df_counties_long, 
                   right=df_counties_wide.drop([f"{col_category} {year}" for year in range(2015, 2019 +1) for col_category in col_categories], axis=1), 
                   how="left", on=cols_to_keep)

In [ ]:
df_counties_long

,FIPS,County_St,Year,Total Number of Services: in Year,Total Medicare Payment Amount: in Year,% ASC Procedures: in Year,% ASC Billing: in Year,Total Number of Services,Total Medicare Payment Amount,tot_ratio,...,Medicare Population Density,Moran I score for ACS billing fraction,County,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY
0,01017,"Chambers, AL",2015,0.0,0.00,0.000000,0.000000,408.0,30064.800000,14.196990,...,14.230610,Non Significant,Chambers,01,017,Chambers,Alabama,01017,AL,AL | CHAMBERS
1,01033,"Colbert, AL",2015,108.0,10404.39,0.000000,0.000000,272.0,37080.230000,16.000000,...,22.681014,Non Significant,Colbert,01,033,Colbert,Alabama,01033,AL,AL | COLBERT
2,01045,"Dale, AL",2015,0.0,0.00,0.000000,0.000000,12.0,405.210000,0.999104,...,17.700437,Non Significant,Dale,01,045,Dale,Alabama,01045,AL,AL | DALE
3,01083,"Limestone, AL",2015,0.0,0.00,0.000000,0.000000,55.0,9515.590000,4.000000,...,29.157261,Non Significant,Limestone,01,083,Limestone,Alabama,01083,AL,AL | LIMESTONE
4,05145,"White, AR",2015,1217.0,48412.57,0.000000,0.000000,1269.0,52190.220000,11.995594,...,15.224018,Non Significant,White,05,145,White,Arkansas,05145,AR,AR | WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4700,21073,"Franklin, KY",2019,0.0,0.00,0.000000,0.000000,114.0,7749.960000,3.910144,...,62.858188,Non Significant,Franklin,21,073,Franklin,Kentucky,21073,KY,KY | FRANKLIN
4701,56021,"Laramie, WY",2019,422.0,79083.86,100.000000,100.000000,1784.0,337949.890001,21.000000,...,6.286729,Non Significant,Laramie,56,021,Laramie,Wyoming,56021,WY,WY | LARAMIE
4702,54041,"Lewis, WV",2019,0.0,0.00,0.000000,0.000000,606.0,26648.230000,4.000000,...,10.524948,Low-Low,Lewis,54,041,Lewis,West Virginia,54041,WV,WV | LEWIS
4703,50027,"Windsor, VT",2019,319.0,12093.61,0.000000,0.000000,1132.0,47825.130000,35.000000,...,14.320922,Low-Low,Windsor,50,027,Windsor,Vermont,50027,VT,VT | WINDSOR


## Set up for summaries and save sums

In [ ]:
# sorted_moran_values = df_counties_wide["Moran I for ASC billing fraction"].unique()
sorted_moran_values = ["High-High","Low-Low","Low-High","High-Low","Non Significant"]  # list out specifically so you can get the order you want
sorted_moran_values_all = sorted_moran_values + ["All"]   #[pd.IndexSlice[:]]  # pd.IndexSlice[:]] represents all

moran_frequencies = df_counties_wide["Moran I for ASC billing fraction"].value_counts()[sorted_moran_values]

In [ ]:
summable_groups = [col for col in df_counties_wide.columns if "total" in col.lower()]
summable_groups = summable_groups + ["Overall Population", "Medicare Beneficiaries Population"]
df_wide_sums = df_counties_wide.groupby("Moran I for ASC billing fraction")[summable_groups].sum()
df_wide_sums = df_wide_sums.assign(Counties=moran_frequencies)
df_wide_sums.loc["All"] = df_wide_sums.sum()

df_wide_sums = df_wide_sums[df_wide_sums.columns[::-1]]  # flip column order left-right to be more logical
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(df_wide_sums)

save_df(df_wide_sums, "df_wide_sums")

,Counties,Medicare Beneficiaries Population,Overall Population,Total Medicare Payment Amount: 2015,Total Number of Services: 2015,Total Medicare Payment Amount: 2016,Total Number of Services: 2016,Total Medicare Payment Amount: 2017,Total Number of Services: 2017,Total Medicare Payment Amount: 2018,Total Number of Services: 2018,Total Medicare Payment Amount: 2019,Total Number of Services: 2019,Total Medicare Payment Amount,Total Number of Services
Moran I for ASC billing fraction,,,,,,,,,,,,,,,
High-High,90,"8,252,843","49,290,013","17,139,184","87,102","18,933,282","106,216","20,196,017","104,668","20,980,333","106,943","21,192,147","116,676","98,440,963","521,605"
High-Low,33,"1,530,091","8,804,718","2,888,148","15,446","3,229,780","15,408","3,008,710","14,722","2,804,513","13,691","2,815,553","14,891","14,746,704","74,158"
Low-High,74,"4,004,628","26,958,805","8,693,092","106,478","7,748,456","90,410","7,584,302","90,625","8,247,523","101,380","8,687,247","103,071","40,960,620","491,964"
Low-Low,139,"3,623,493","19,364,704","5,561,819","95,757","4,296,993","65,244","3,946,056","59,105","4,005,969","61,639","4,649,317","72,170","22,460,154","353,915"
Non Significant,605,"25,346,776","163,845,060","57,438,585","472,764","55,995,168","396,368","58,543,009","428,827","60,913,491","429,358","63,638,158","434,911","296,528,411","2,162,228"
All,941,"42,757,830","268,263,300","91,720,829","777,547","90,203,679","673,646","93,278,093","697,947","96,951,828","713,011","100,982,422","741,719","473,136,851","3,603,870"


In [ ]:
!ls outputs/2022_11_20-RSY-geospatial_ENT_analysis_v03 

df_counties_summary_clean.csv	df_wide_sums.csv
df_counties_summary_clean.xlsx	df_wide_sums.xlsx


## Create summary data by Moran category

In [ ]:
col_categories = ["Total Number of Services","Total Medicare Payment Amount", "% ASC Procedures", "% ASC Billing" ]

df_counties_with_slope = df_counties_wide.copy()
# Calculate regression and sum and mean from individual year later
for col_category in col_categories:
    new_df = df_counties_with_slope[ [col_category + ": " + str(yr) for yr in years] ].apply(calc_regression,axis=1, result_type="expand", args=(years,) )
    df_counties_with_slope[[col_category+": "+new_col for new_col in new_df.columns ]]=new_df
# To simplify, drop info for specific years unless it was "Mean" and "Slope" col_categories we just added
for col_category in col_categories:
    df_counties_with_slope = df_counties_with_slope.drop(columns=[col for col in df_counties_with_slope.columns if col_category in col and "Mean" not in col and "Slope" not in col])


df_counties_summary_dict = {}   # create a dict we will concatenate into a df later
# Options: 	[count, mean, std, min, 25%, 50%, 75%, max] assuming default percentiles argument
cols_to_show = ["10%","mean","90%"]
for possible_Moran_value in sorted_moran_values:
    df_counties_summary_dict[possible_Moran_value] = df_counties_with_slope[df_counties_with_slope["Moran I for ASC billing fraction"]==possible_Moran_value].describe(percentiles=[.1,.25,.5,.75,.9]).loc[cols_to_show]
df_counties_summary_dict["All"] = df_counties_with_slope.describe(percentiles=[.1,.25,.5,.75,.9]).loc[cols_to_show]

df_counties_summary = pd.concat(df_counties_summary_dict.values(), axis=0, keys=df_counties_summary_dict.keys())
for possible_Moran_value in sorted_moran_values:
    df_counties_summary.loc[(possible_Moran_value,cols_to_show[0]), "N"] = moran_frequencies[possible_Moran_value]

# Reorder into the sorted order we set above
df_counties_summary = df_counties_summary.loc[sorted_moran_values_all]



## Create a more presentable format
Select out only the columns you want and rename the columns

In [ ]:
df_counties_long.columns

Index(['FIPS', 'County_St', 'Year', 'Total Number of Services: in Year',
       'Total Medicare Payment Amount: in Year', '% ASC Procedures: in Year',
       '% ASC Billing: in Year', 'Total Number of Services',
       'Total Medicare Payment Amount', 'tot_ratio', '% ASC Procedures',
       '% ASC Billing', 'Beneficiaries with Part A and Part B', 'Average Age',
       'Percent Male', 'Percent Non-Hispanic White',
       'Percent African American', 'Percent Hispanic',
       'Percent Eligible for Medicaid', 'Average HCC Score',
       'Hospital Readmission Rate',
       'Emergency Department Visits per 1000 Beneficiaries',
       'Procedures Per Capita Standardized Costs',
       'Procedure Events Per 1000 Beneficiaries', 'metro', 'pct_poverty',
       'median_house_income', 'pop', '2013_Rural_urban_cont_code',
       'Pct_wthout_high_diploma', 'Pct_wth_high_diploma', 'Pct_wth_some_coll',
       'Pct_wth_coll_degree', 'unemployment', 'pct_uninsured', 'fibro',
       'tabacco', 'obesity'

'% ASC Billing:'

In [ ]:
key_cols={
    'Total Number of Services: Mean': 'Number of Services: Annual' ,
    'Total Number of Services: Slope': 'Number of Services: Yearly Change' ,
    'Total Medicare Payment Amount: Mean' : "Medicare Payment: Annual",
    'Total Medicare Payment Amount: Slope' : "Medicare Payment: Yearly Change",
    '% ASC Billing: Mean': '% ASC Billing',
    '% ASC Billing: Slope': '% ASC Billing: Yearly Change',
    "% ASC Procedures: Mean": "% ASC Procedures",
    "% ASC Procedures: Slope": "% ASC Procedures: Yearly Change",
    "Average Age": "Average Age (years)",
    'Percent Male': "% Male",
    'Percent Non-Hispanic White': "% Non-Hispanic White",
    'Percent African American': "% African American",
    'Percent Hispanic': "% Hispanic",
    'Percent Eligible for Medicaid': "% Eligible for Medicaid",
    'pct_poverty': "% Poverty",
    'median_house_income': "Median Household Income",
    "Pct_wthout_high_diploma": "% without High School Graduation",
    'unemployment': "Unemployment Rate",
    'pct_uninsured': "% Uninsured",
    'tabacco': "% Tobacco Use",
    'obesity': "% Obesity",
    #"Asthma": "% with Asthma",
    '2013_Rural_urban_cont_code': "RUCA",
    'pop': "Overall Population",
    'Beneficiaries with Part A and Part B': "Medicare Beneficiaries Population",
    'Population Density': "Overall Population Density",
    'Medicare Population Density': "Medicare Population Density",
}
df_counties_summary_clean = df_counties_summary[key_cols.keys()]
df_counties_summary_clean = df_counties_summary_clean.rename(columns=key_cols).transpose()

with pd.option_context('display.float_format', '{:,.2f}'.format):
    display( df_counties_summary_clean )

save_df(df_counties_summary_clean, "df_counties_summary_clean")

High-High                          Low-Low  \
                                         10%       mean        90%        10%   
Number of Services: Annual             54.38   1,159.12   1,185.06       8.72   
Number of Services: Yearly Change    -115.49      66.53      54.78    -193.22   
Medicare Payment: Annual           16,018.24 218,757.69 439,851.70     798.98   
Medicare Payment: Yearly Change   -22,898.25  11,281.08  26,467.42 -11,950.69   
% ASC Billing                          37.72      69.28      90.97       0.00   
% ASC Billing: Yearly Change           -4.89       1.44       8.11       0.00   
% ASC Procedures                       17.44      51.27      80.33       0.00   
% ASC Procedures: Yearly Change        -6.74       1.34       9.26       0.00   
Average Age (years)                    71.00      72.41      74.00      68.80   
% Male                                 42.93      44.69      46.80      44.12   
% Non-Hispanic White                   63.48      80.69      92.80      77.03   
% African American                      1.32       8.87      19.75       0.31   
% Hispanic                              0.93       5.83      14.06       0.29   
% Eligible for Medicaid                 9.93      16.61      23.62      15.45   
% Poverty                               6.90      12.54      19.14       8.42   
Median Household Income            47,690.34  64,248.34  90,205.08  42,085.80   
% without High School Graduation        6.90      11.62      15.05       5.50   
Unemployment Rate                       3.60       4.76       5.97       2.88   
% Uninsured                             5.90      10.75      16.56       4.64   
% Tobacco Use                           5.00       7.65      10.04       8.20   
% Obesity                              13.00      19.13      25.12      13.08   
RUCA                                    1.00       1.92       3.00       1.00   
Overall Population                102,551.02 547,666.81 914,269.40  36,255.12   
Medicare Beneficiaries Population  15,823.74  91,698.25 143,264.70   8,175.44   
Overall Population Density            129.82     786.57   1,906.68      48.11   
Medicare Population Density            20.49     134.04     265.73      10.74   

                                                         Low-High             \
                                        mean        90%       10%       mean   
Number of Services: Annual            509.23   1,402.84      3.18   1,329.63   
Number of Services: Yearly Change     -36.53      96.40    -97.49       5.62   
Medicare Payment: Annual           32,316.77  82,467.87    535.71 110,704.38   
Medicare Payment: Yearly Change    -1,522.32   5,963.24 -6,221.68     658.62   
% ASC Billing                           1.12       1.83      0.00       2.74   
% ASC Billing: Yearly Change            0.03       0.00     -1.16      -0.51   
% ASC Procedures                        0.45       0.37      0.00       0.93   
% ASC Procedures: Yearly Change        -0.05       0.00     -0.13      -0.22   
Average Age (years)                    70.37      72.00     69.66      71.49   
% Male                                 45.95      47.51     42.75      45.13   
% Non-Hispanic White                   88.03      96.23     53.49      77.02   
% African American                      4.13      10.02      0.65      12.98   
% Hispanic                              1.22       2.31      0.57       5.69   
% Eligible for Medicaid                23.85      33.66     10.35      19.00   
% Poverty                              13.96      20.09      8.28      14.25   
Median Household Income            54,240.51  68,665.08 41,664.62  59,892.96   
% without High School Graduation        9.98      14.50      7.43      12.56   
Unemployment Rate                       4.58       6.32      3.23       4.67   
% Uninsured                             7.27      11.10      6.70      11.58   
% Tobacco Use                          11.62      16.52      5.66       8.58   
% Obes

# Graph